**Analyse et segmentation de clientèle d'un magasin avec campagnes de marketing**
_Jules EXBRAYAT & Abdenour MADANI_

# Présentation

## Problématique

Quel est le client type de l'entreprise ?

Peut-on créer des groupes de clients, afin de cibler efficacement les potentiels clients et personnaliser la campagne marketing ?

## Jeu de données

In [ ]:
# todo : ajouter une description / une partie pour parler du jeu de données
# "Ce jeu de données contient le profil de plusieurs clients d'une enseigne de grande distribution" \
# "Il contient les données de juillet 2012 à juillet 2014" \
# "Nous disposons des ventes en Vin, etc"

### Lecture des données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.genmod.generalized_linear_model

In [ ]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv("marketing_campaign.csv", sep="\t", index_col="ID", parse_dates=True)

### Présentation des variables

In [ ]:
df.head()

In [ ]:
print(f"Il y a {df.shape[1]} variables et {df.shape[0]} individus.")

#### Variables

- ID: identifiant du client
- Year_Birth: numérique, année de naissance du client
- Education: qualitative, niveau d'éducation
- Marital_Status: qualitative, statut marital
- Income: numérique, revenu annuel en $
- Kidhome: numérique, nombre d'enfants en bas-âge
- Teenhome: numérique, nombre d'enfants adolescents
- Dt_Customer: date, date à laquelle le client s'est inscrit
- Recency: numérique, nombre de jours depuis le dernier achat
- Complain: catégorique, est-ce que le client s'est plaint les 2 dernières années (0 ou 1)

#### Products

- MntWines: numérique, argent dépensé les 2 dernières années en vin
- MntFruits: numérique, argent dépensé les 2 dernières années en fruits
- MntMeatProducts: numérique, argent dépensé les 2 dernières années en viande
- MntFishProducts: numérique, argent dépensé les 2 dernières années en poisson
- MntSweetProducts: numérique, argent dépensé les 2 dernières années en bonbons / gâteaux
- MntGoldProds: numérique, argent dépensé les 2 dernières années en bijoux / or

#### Promotion

- NumDealsPurchases: numérique, nombre d'achats effectués avec une promotion
- AcceptedCmp1: catégorique, le client a-t-il acheté durant la campagne promotionnelle numéro 1 (1 s'il a acheté, 0 sinon)
- AcceptedCmp2: pareil pour la campagne numéro 2
- AcceptedCmp3: pareil pour la campagne numéro 3
- AcceptedCmp4: pareil pour la campagne numéro 4
- AcceptedCmp5: pareil pour la campagne numéro 5
- Response: catégorique, 1 si le client a acheté durant la dernière campagne, 0 sinon (potentielle variable à prédire)

#### Place

- NumWebPurchases: numérique, nombre d'achats effectués sur le site Internet
- NumCatalogPurchases: numérique, nombre d'achats effectués via le catalogue
- NumStorePurchases: numérique, nombre d'achats effectués en magasin
- NumWebVisitsMonth: numérique, nombre de visites sur le site Internet le dernier mois


Nous séparons les variables numériques des variables catégoriques pour plus de commodités.

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Kidhome",
    "Teenhome",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Complain",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

Nous convertissons les variables catégoriques en type `category`. (Nous les convertissons au préalable en type `string` car cela facilite l'affichage de la légende avec Matplotlib et Seaborn)

In [ ]:
df[var_categoriques] = df[var_categoriques].astype(str).astype("category")

Nous convertissons les variables au format date.

In [ ]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format="%d-%m-%Y")

In [ ]:
df.info()

Nous avons 9 variables catégoriques, 18 variables quantitatives (dont 17 entières) ainsi qu'une variable de type date.

In [ ]:
# todo: inclure "Découverte" et "Nettoyage" dans une grande partie "Analyse exploratoire des données" ?

# Découverte des données

## Analyse univariée

In [ ]:
df[var_numeriques].describe()

In [ ]:
df[var_categoriques].describe()

In [ ]:
# todo: commenter

## Visualisation

### Variables numériques

In [ ]:
for var in var_numeriques:
    _, ax = plt.subplots(1, 2, figsize=(8, 2))
    sns.boxplot(df[var], width=0.25, ax=ax[0])
    sns.histplot(df[var], kde=True, ax=ax[1])
    plt.show()

In [ ]:
# todo: commenter les distributions et boxplots

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(
    df[var_numeriques].corr()[df[var_numeriques].corr().abs() > 0.5],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

In [ ]:
# todo: commenter la matrice de corrélation

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].set_title("Nombre de valeurs présentes")
df.notna().sum()[var_numeriques].plot.barh(ax=ax[0])

ax[1].set_title("Valeurs manquantes")
sns.heatmap(df[var_numeriques].isna(), cbar=False, ax=ax[1])

Nous observons qu'il n'y a quasiment pas de valeurs manquantes parmi les variables numériques.

### Variables catégoriques

In [ ]:
for var in var_categoriques:
    if df[var].nunique() > 3:
        sns.histplot(y=df[var])
    else:
        plt.figure(figsize=(4, 2))
        sns.histplot(df[var], shrink=0.3)
    plt.show()

In [ ]:
df[
    df[var_categoriques].columns[df[var_categoriques].nunique() > 3]
].value_counts().plot(kind="bar")

In [ ]:
# todo: commenter

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].set_title("Nombre de valeurs présentes")
df.notna().sum()[var_categoriques].plot.barh(ax=ax[0])

ax[1].set_title("Valeurs manquantes")
sns.heatmap(df[var_categoriques].isna(), cbar=False, ax=ax[1])

Nous observons l'absence de valeurs manquantes parmi les variables catégoriques.

### Dates

In [ ]:
# todo

In [ ]:
df["Dt_Customer"].hist(bins=50)

# Nettoyage des données

## Données parasites


### Variables

Dans le jeu de données, 2 colonnes sont non documentées et contiennent des valeurs constantes :
- `Z_CostContact`
- `Z_Revenue`

**Variable `Z_CostContact`**

In [ ]:
print(
    f'Il y a {np.sum(df["Z_CostContact"] == 3)} / {len(df)} individus pour lesquelles la variable Z_CostContact vaut 3.'
)

In [ ]:
df["Z_CostContact"].hist()

**Variable `Z_Revenue`**

In [ ]:
print(
    f'Il y a {np.sum(df["Z_Revenue"] == 11)} / {len(df)} individus pour lesquelles la variable Z_Revenue vaut 11.'
)

In [ ]:
df["Z_Revenue"].hist()

Nous décidons de les supprimer, car elles sont parasites et ne fournissent pas d'information.

In [ ]:
df.drop(columns=["Z_CostContact", "Z_Revenue"], inplace=True)

### Valeurs

#### Statut marital

In [ ]:
sns.histplot(y=df["Marital_Status"])

Parmi les situations maritales, on observe "absurde" et "YOLO" (You Only Live Once, _on ne vit qu'une fois_).

Comme on ne peut pas déduire ce que veut dire "absurde" dans ce contexte, on se décide de supprimer l'individu.
De même, on suppose que la personne ayant répondu "YOLO", donc ayant répondu de façon humoristique, doit être supprimée du jeu de données.

In [ ]:
df.drop(index=df.loc[df["Marital_Status"] == "YOLO"].index, inplace=True)
df.drop(index=df.loc[df["Marital_Status"] == "Absurd"].index, inplace=True)

In [ ]:
# On retire les options "YOLO" et "Absurd" de nos variables catégoriques.
df["Marital_Status"] = df["Marital_Status"].cat.remove_categories(["YOLO", "Absurd"])

In [ ]:
sns.histplot(y=df["Marital_Status"])

## Données dupliquées

### Individus

In [ ]:
sum(df.duplicated())

Nous observons qu'il y a 182 lignes qui sont dupliquées.

Nous pouvons supposer que les données ont été fusionnées à partir de plusieurs sources, et que ces sources avaient des individus en commun : nous décidons donc de supprimer les doublons.

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
sum(df.duplicated())

## Variables pas assez informatives

In [ ]:
# todo: dire que la variable "Complain" (si le client s'est plaint ou non), n'apporte pas assez d'information pour être gardée, car il y a moins de 1% des observations qui ont une classe autre que "0", et qu'en plus ce n'est pas central à notre étude

In [ ]:
# todo: essayer quand même une visualisation des individus s'étant plaint par rapport à nos variables cibles, au cas où...

In [ ]:
sns.histplot(df["Complain"], shrink=0.5)

In [ ]:
print(
    f'Il y a {100 * df["Complain"].astype(int).sum() / len(df["Complain"]):.2f}% des individus qui se sont plaints.'
)

## Standardisation

### Modalités de variables catégoriques

In [ ]:
sns.histplot(y=df["Marital_Status"])

Sans plus d'information, "Alone" est une autre formulation de "Single", on décide donc de remplacer "Alone" par "Single".

In [ ]:
df["Marital_Status"].replace("Alone", "Single", inplace=True)

In [ ]:
# On retire l'option "Alone" de nos variables catégoriques.
df["Marital_Status"] = df["Marital_Status"].cat.remove_categories(["Alone"])

In [ ]:
sns.histplot(y=df["Marital_Status"])

## Valeurs extrêmes

### Income

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

In [ ]:
# todo: texte pour commenter

In [ ]:
df["Income"][df["Income"] > 200_000]

Pour nous, il s'agit d'une valeur aberrante à supprimer du jeu de données car "666 666" ne semble pas être une valeur réelle (6 fois le nombre 6...).

In [ ]:
df.drop(index=df[df["Income"] > 200_000].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

In [ ]:
# todo: est-ce qu'on retire les valeurs au-dessus de 120 000 ? comment on le justifie ?
# => ce serait plutôt des individus ATYPIQUES, à enlever mais garder de côté pour l'analyse a posteriori

In [ ]:
df.drop(index=df[df["Income"] > 120_000].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Income"], width=0.25, ax=ax[0])
sns.histplot(df["Income"], kde=True, ax=ax[1])
plt.show()

### Year Date

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Year_Birth"], width=0.25, ax=ax[0])
sns.histplot(df["Year_Birth"], kde=True, ax=ax[1])
plt.show()

In [ ]:
df[df["Year_Birth"] < 1920]

Comme les personnes les plus âgées au monde ont très rarement plus de 100 ans, et que nous sommes en 2023, on peut sans problème estimer que les trois dates de naissance avant 1920 sont erronées.

Pour les deux années 1893 et 1899, on peut s'imaginer que les personnes ont fait une faute de frappe, et voulaient écrire, respectivement, 1983 et 1989.

In [ ]:
df.loc[df["Year_Birth"] == 1893, "Year_Birth"] = 1983
df.loc[df["Year_Birth"] == 1899, "Year_Birth"] = 1989

Pour l'année 1900, il est difficile d'imaginer une faute de frappe, il s'agit peut-être de la valeur par défaut.
On peut tout simplement supprimer la ligne correspondante.

In [ ]:
df.drop(index=df.loc[df["Year_Birth"] == 1900, "Year_Birth"].index, inplace=True)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(df["Year_Birth"], width=0.25, ax=ax[0])
sns.histplot(df["Year_Birth"], kde=True, ax=ax[1])
plt.show()

## Valeurs manquantes

In [ ]:
sns.heatmap(
    df.isna(), cbar=False
)  # todo: mettre les graphiques à côté (horizontalement)

In [ ]:
sns.displot(
    data=df.isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill",
    aspect=1.25,
)

In [ ]:
plt.title("Nombre de valeurs manquantes pour chaque variable")
df.isna().sum().plot.barh()

In [ ]:
# todo : commenter

In [ ]:
# TODO: supprimer valeurs manquantes OU remplacer par la moyenne ?

In [ ]:
# todo: expliquer POURQUOI ces données sont manquantes (notre hypothèse du moins), et expliquer notre démarche en conséquence

### Remplacement par la moyenne

In [ ]:
df["Income"][df["Income"].isna()]

In [ ]:
df["Income"].fillna(df["Income"].mean(), inplace=True)

In [ ]:
# Vérification du bon remplacement des valeurs manquantes
df["Income"].isna().sum()

# Création de variables

In [ ]:
# todo : faire une var somme des achats sur le web
# todo : faire un client type par type d'achat (web / magasin) => est-ce que les gens qui achètent en web / magasin sont différents ?
# todo : créer une variable duréeClient qui dit depuis quand le client est inscrit

In [ ]:
# todo: commenter la démarche

In [ ]:
df["NbAcceptedCampaigns"] = 0

for i in range(1, 6):
    df["NbAcceptedCampaigns"] += df[f"AcceptedCmp{i}"].astype(int)

In [ ]:
sns.histplot(df["NbAcceptedCampaigns"], discrete=True)

In [ ]:
df["HasAcceptedCampaigns"] = df["NbAcceptedCampaigns"] > 0

In [ ]:
sns.histplot(df["HasAcceptedCampaigns"].astype(str))

In [ ]:
df_clients = df[df["HasAcceptedCampaigns"]]
df_not_clients = df[~df["HasAcceptedCampaigns"]]

In [ ]:
# todo: créer une variable qui additionne le nombre d'enfants Kids et Teen (mais avant cela, vérifier si par exemple Kids et Teen apportent des informations différentes => si les deux sont complémentaires, on les laisse / sinon on les supprime au profit de notre nouvelle variable)

In [ ]:
df[["Kidhome", "Teenhome"]].value_counts().plot(kind="bar")

In [ ]:
df["NbChildren"] = df["Kidhome"] + df["Teenhome"]

In [ ]:
sns.histplot(df["NbChildren"])

In [ ]:
# todo: créer une variable "NbAcceptedCampaignsWithResponse", qui contient donc la 6ème campagne marketing => l'objectif est de clusteriser les gens en fonction de "à quel point ils sont marketables" => c'est-à-dire, on présente nos résultats en disant sur qui il vaut mieux se concentrer pour faire des pubs

In [ ]:
# todo: créer une variable "RevenuePerClient" => le revenu rapporté par chaque client => ce serait plutôt juste une variable type "KPI" à afficher dans un dashboard non ?

In [ ]:
# todo: de même, créer une variable "RevenueWinePerClient" => le revenu de vin pour chaque client, et faire de même pour chaque produit => par la suite on pourra faire des représentations "quel client rapporte le plus de vin"

# Visualisation

In [ ]:
# todo: renommer cette partie ? / mettre un texte introductif pour expliquer la démarche ("Une fois les données nettoyées, on les explore de nouveau, plus en profondeur")

In [ ]:
# todo: organiser en plusieurs sous-parties

In [ ]:
# todo: choisir les représentations les plus pertinentes à montrer ci-dessous

In [ ]:
sns.histplot(
    df,
    x="Income",
    hue="NbAcceptedCampaigns",
    kde=True,
    stat="density",
    common_norm=False,
)

In [ ]:
sns.histplot(
    df,
    x="Income",
    hue="NbAcceptedCampaigns",
    kde=True,
    stat="probability",
    common_norm=False,
)

In [ ]:
sns.displot(df, x="Income", hue="NbAcceptedCampaigns", kind="kde", common_norm=False)

In [ ]:
# todo: faire la même chose en boxplots !

In [ ]:
sns.histplot(df, x="Income", hue="HasAcceptedCampaigns", kde=True)

In [ ]:
sns.histplot(df, x="Income", hue="Response", kde=True)

### Année naissance (test)

In [ ]:
sns.histplot(df, x="Year_Birth", hue="Response", kde=True)

### todo

In [ ]:
sns.histplot(
    data=df, x="Education", hue="HasAcceptedCampaigns", multiple="dodge", shrink=0.8
)

In [ ]:
sns.histplot(
    data=df,
    x="NbChildren",
    hue="Response",
    multiple="dodge",
    discrete=True,
    shrink=0.4,
)

In [ ]:
sns.histplot(
    data=df,
    x="NbChildren",
    hue="HasAcceptedCampaigns",
    multiple="dodge",
    discrete=True,
    shrink=0.4,
)

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(
    df.corr()[df.corr().abs() > 0.5],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

## Régression polynomiale

### Premier exemple

In [ ]:
sns.scatterplot(data=df, x="Income", y="MntWines", color="black")

In [ ]:
sns.scatterplot(x=df["Income"] ** 2, y=df["MntWines"], color="black")

In [ ]:
sns.regplot(
    x=df["Income"],
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

In [ ]:
sns.regplot(
    x=df["Income"] ** 2,
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

In [ ]:
sns.regplot(
    x=df["Income"] ** 3,
    y=df["MntWines"],
    scatter_kws={"color": "black"},
    line_kws={"color": "red"},
)

### Afficher la courbe

In [ ]:
# Régression linéaire basique
from sklearn.linear_model import LinearRegression

In [ ]:
X = np.array(df[["Income"]] ** 2)
y = np.array(df[["MntWines"]])

In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(df["Income"], reg.predict(X), label="Régression polynomiale", color="red")

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

### Polynomial Features

In [ ]:
# Pour la régression polynomiale
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

Si on donne une variable $V$, PolynomialFeatures crée les variables
- $1$
- $V$
- $V^2$

Si on donne les variables $V$ et $W$, PolynomialFeatures crée les variables
- $1$
- $V$
- $W$
- $V \times W$
- $V^2$
- $W^2$

(attention à l'explosion combinatoire du nombre de variables !)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
reg = LinearRegression().fit(X_poly, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(
    df["Income"], reg.predict(X_poly), label="Régression polynomiale", color="red"
)

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

### Attention au sur-apprentissage

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=10)
X_poly = poly.fit_transform(X)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
reg = LinearRegression().fit(X_poly, y)

In [ ]:
# Tracer les données et la courbe de régression polynomiale
plt.scatter(df["Income"], y, label="Données", color="black")
plt.scatter(
    df["Income"], reg.predict(X_poly), label="Régression polynomiale", color="red"
)

plt.legend(loc="best")
plt.title("Régression polynomiale d'un cube de données synthétiques")

plt.show()

## todo

In [ ]:
sns.scatterplot(data=df, x="Income", y="MntMeatProducts", hue="Marital_Status")

In [ ]:
sns.scatterplot(data=df, x="Income", y="MntWines")

In [ ]:
sns.histplot(data=df, x="Education", hue="Response", multiple="dodge", shrink=0.8)

In [ ]:
sns.displot(
    data=df,
    x="Marital_Status",
    hue="Response",
    col="Kidhome",
    multiple="dodge",
    shrink=0.8,
)

# Tests d'hypothèse

## ANOVA

### Problématique

Nous allons tester l'indépendance entre la variable `Response` (catégorique, binaire) et `Income` (quantitative continue), pour répondre à la question : "le revenu influence-t-il la réponse aux campagnes marketing ?".

In [ ]:
sns.boxplot(df, x="Income", y="Response")

On cherche à déterminer si les moyennes des groupes sont significativement différentes. On pose donc :

$H_0$ : Les moyennes de chaque groupe sont égales si la p-value $> 5\%$
$H_1$ : Les moyennes de chaque groupe ne sont pas toutes égales si la p-value $< 5\%$


### Hypothèses

1) l’indépendance entre les échantillons de chaque groupe
2) l’égalité des variances que l’on peut verifier avec un test de Bartlett.
3) la normalité des résidus avec un test de Shapiro.

#### Indépendance

#### Egalité des variances

In [ ]:
from scipy.stats import bartlett

Voici la variance de chaque groupe :

In [ ]:
df.groupby("Response")["Income"].agg("var")

Nous allons effectuer un test de Bartlett pour vérifier l'égalité des variances.

$H_0$ : Les variances de chaque groupe sont égales si p-value $> 5\%$
$H_1$ : Les variances de chaque groupe ne sont pas toutes égales $< 5\%$


In [ ]:
bartlett(
    df["Income"][df["Response"] == "0"],
    df["Income"][df["Response"] == "1"],
)

Notre p-value est inférieure à 5%, donc les variances ne sont pas toutes égales au (risque de 5%).

#### Normalité des résidus

Nous utilisons le test de Shapiro-Wilk pour vérifier la normalité des résidus.

H0 : Les résidus suivent une loi normale si p-value > 5%
H1 : Les résidus ne suivent pas une loi normale si p-value < 5%

In [ ]:
from scipy.stats import shapiro
from statsmodels.formula.api import ols

model = ols("Income ~ Response", data=df).fit()
shapiro(model.resid)

Comme la p-value est inférieure à 5%, les résidus ne suivent pas une loi normale (au risque de 5%).

#### Test d'ANOVA

H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
import statsmodels.api as sm

anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

Comme aucune hypothèse n'est vérifiée, le test n'est pas fiable.

In [ ]:
# todo: tenter le GLS

# Transformation des données

In [ ]:
df_transforme = df.copy()

## Mise à l'échelle (réduction ?)

### Explication

In [ ]:
# todo: expliquer pourquoi on a envie de le faire

### Sélection des variables

In [ ]:
df[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

In [ ]:
# todo(prof): est-ce que notre choix de variables est bon ?
var_a_scaler = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
]

In [ ]:
df["Income"].hist()

In [ ]:
(df["Income"] / df["Income"].std()).hist()

### Transformation

In [ ]:
for var in var_a_scaler:
    df_transforme[var] = df[var] / df[var].std()
    df_transforme[var] -= df_transforme[var].min()

### Conclusion

In [ ]:
df_transforme[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

Les changements sur la matrice de corrélation sont de l'ordre de $10^{-15}$, donc extrêmement négligeables.

In [ ]:
sns.heatmap(df_transforme[var_numeriques].corr() - df[var_numeriques].corr())

## Normalisation

### Explication

Pour normaliser les données, nous allons utiliser la transformation de Box-Cox, définie $\forall x > 0, $ comme ci-dessous :
$B(x, \lambda) = \begin{cases} \frac{x^{\lambda} - 1}{\lambda} & \text{  si } \lambda \neq 0 \\ \log(x) & \text{  si } \lambda = 0 \end{cases}$

Cette transformation est à appliquer à une variable (strictement positive), en ajustant le $\lambda$ pour maximiser la normalité.

Nous allons utiliser la librairie `scipy.stats.boxcox` qui estime le meilleur paramètre $\lambda$.

### Sélection des variables

In [ ]:
df[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

In [ ]:
# todo(prof): faut-il inclure la distribution uniforme Recency ?
# todo(prof): est-ce que notre choix de variables est bon ?
var_a_normaliser = [
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
]

### Transformation

In [ ]:
sns.histplot(df["MntWines"], kde=True)

In [ ]:
from scipy.stats import boxcox

In [ ]:
boxcox_lambdas = {}  # on garde les lambdas, pour la transformation inverse

In [ ]:
for var in var_a_normaliser:
    var_strict_positif = df[var] + df[var].min() + 1

    var_apres_boxcox, l = boxcox(var_strict_positif)

    df_transforme[var] = var_apres_boxcox
    boxcox_lambdas[var] = l

In [ ]:
# Tuto : comment récupérer la fonction initiale
# (il faut avoir récupéré le paramètre "l" lambda)

# from scipy.special import inv_boxcox
# initial = inv_boxcox(incbox, l)
# initial = pd.DataFrame(initial)
# sns.histplot(initial, bins=50, kde=True)

In [ ]:
sns.histplot(df_transforme["MntWines"], bins=50, kde=True)

In [ ]:
df_transforme[var_numeriques].hist(figsize=(12, 12), bins=30)
plt.show()

### Conclusion

In [ ]:
# todo: analyser les résultats, notamment l'impact sur la matrice de corrélation (du coup, mettre ci-dessous la matrice de corrélation pré-transformation et comparer)

In [ ]:
sns.heatmap(
    df_transforme[var_numeriques].corr() - df[var_numeriques].corr(),
    cmap="BrBG",
    vmin=-1,
    vmax=1,
)

In [ ]:
df_transforme[var_numeriques].corr() - df[var_numeriques].corr()

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Matrice de corrélation des variables numériques avant transformation")
sns.heatmap(
    df[var_numeriques].corr()[df[var_numeriques].corr().abs() > 0.5],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

In [ ]:
plt.figure(figsize=(12, 12))
plt.title("Matrice de corrélation des variables numériques après transformation")
sns.heatmap(
    df_transforme[var_numeriques].corr()[
        df_transforme[var_numeriques].corr().abs() > 0.5
    ],
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

## Variables catégoriques

### Encodage

In [ ]:
# todo: One Hot Encoding, Target Encoding, etc

# Analyse multi-variée

## Clustering

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df_transforme[var_numeriques]),
    columns=df[var_numeriques].columns,
)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k3 = KMeans(n_clusters=3)
k4 = KMeans(n_clusters=4)

In [ ]:
k3.fit(df_apres_scale[var_numeriques])
k4.fit(df_apres_scale[var_numeriques])

In [ ]:
df["cluster"] = k3.labels_

In [ ]:
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

In [ ]:
df[var_numeriques + ["cluster"]]

In [ ]:
df["cluster"] = k4.labels_
sns.histplot(df["cluster"])

In [ ]:
sns.histplot(df, x="Income", hue="cluster", kde=True)

In [ ]:
sns.scatterplot(df, x="Year_Birth", y="Income", hue="cluster")

## Analyse en Composantes Principales (ACP)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# todo: centrer / réduire

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
df_apres_scale = pd.DataFrame(
    scaler.fit_transform(df[var_numeriques]), columns=df[var_numeriques].columns
)

In [ ]:
df_apres_scale

In [ ]:
# for var in var_numeriques:
#     _, ax = plt.subplots(1, 2, figsize=(8, 2))
#     sns.boxplot(df_apres_scale[var], width=0.25, ax=ax[0])
#     sns.histplot(df_apres_scale[var], kde=True, ax=ax[1])
#     plt.show()

In [ ]:
acp = PCA(random_state=0)

In [ ]:
acp.fit(df_apres_scale)

In [ ]:
variance_expliquee = pd.Series(
    acp.explained_variance_ratio_, index=df[var_numeriques].columns
)

In [ ]:
variance_expliquee

In [ ]:
variance_expliquee.plot.barh()

In [ ]:
df_acp = pd.DataFrame(acp.fit_transform(df_apres_scale), index=df.index)

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1)

### Cercle de corrélations

In [ ]:
from mlxtend.plotting import plot_pca_correlation_graph

In [ ]:
_, correlation_matrix = plot_pca_correlation_graph(
    df_apres_scale,
    df_apres_scale.columns,
    X_pca=df_acp.iloc[:, :2],
    explained_variance=acp.explained_variance_[:2],
    dimensions=(1, 2),
)

In [ ]:
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="BrBG",
    linewidths=0.5,
    vmax=1,
    vmin=-1,
)

### Test clusters

In [ ]:
df_acp["clusterk3"] = k3.labels_
df_acp["clusterk4"] = k4.labels_

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk3")

In [ ]:
df_acp.head()

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=2, hue=df["Response"])

In [ ]:
sns.scatterplot(df_acp, x=0, y=1, hue="clusterk4")

## Analyse Factorielle des Correspondances (AFC)

In [ ]:
# todo

## Analyse des Correspondances Multiples (ACM)

In [ ]:
# todo

# Régressions

In [ ]:
from sklearn.model_selection import train_test_split

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

In [ ]:
X = df[var_numeriques]

In [ ]:
y = df[["Response"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
log_reg = LogisticRegression(random_state=0)

In [ ]:
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=log_reg.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=log_reg.classes_)
disp.plot()

In [ ]:
# todo: commenter

## Régression PLS

In [ ]:
# todo: refaire la régression PLS mais PAS sur une variable catégorique (erreur ici)

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
# Initialisation de l'objet PLSRegression avec 2 composantes PLS
pls = PLSRegression(n_components=2)

In [ ]:
# Apprentissage du modèle sur les données
pls.fit(df_transforme[var_numeriques], df_transforme["Response"])

In [ ]:
# Prédiction de la variable cible sur de nouvelles données
y_pred = pls.predict(df_transforme[var_numeriques])

In [ ]:
# Evaluation de la performance du modèle
r2 = pls.score(df_transforme[var_numeriques], df_transforme["Response"].astype(int))

In [ ]:
r2

## Régression linéaire simple

Modèle simple : une variable à expliquer Y et une seule variable explicative X.  

$$y_i = \beta_0 + \beta_1 X_i + \epsilon_i$$

Hypothèses à vérifier pour la régression linéaire simple :  

1) il existe une corrélation linéaire entre X et Y

1) la distribution de l’erreur ε est indépendante de la variable X (exogénéité)

2) l'erreur suit une loi normale centrée i.e. E(ε) = 0

3) l’erreur est de variance constante (homoscédasticité)
i.e Var(εi) = s, s une constante

4) les erreurs sont indépendantes (absence d'autocorrélation)
i.e. Cov(εi, εj) = 0, pour tout i, j

In [ ]:
X = np.array(df_transforme["Income"])
Y = np.array(df_transforme["NumStorePurchases"])

### Hypothèse 1 : corrélation linéaire

In [ ]:
from scipy.stats import pearsonr

In [ ]:
print(
    "Le coefficient de corrélation linéaire entre X et Y vaut",
    pearsonr(X, Y)[0],
    "la pvalue associée vaut",
    pearsonr(X, Y)[1],
    "il existe donc bien une relation linéaire entre X et Y.",
)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
model = LinearRegression().fit(X_train.reshape(-1, 1), Y_train)
Y_train_hat = model.predict(X_train.reshape(-1, 1))

In [ ]:
plt.scatter(X_train, Y_train, color="black")
plt.plot(X_train, Y_train_hat, color="red")
plt.title("Régression linéaire du nombre d'achats en magasin en fonction du revenu")

In [ ]:
print(model.intercept_, model.coef_, model.score(X_train.reshape(-1, 1), Y_train))

Equation de régression :

$$y_i = 0.46 + 2.21 * x_i$$

### Hypothèse 2 : exogénéité

In [ ]:
residuals = Y_train - Y_train_hat

plt.scatter(X_train, residuals)

In [ ]:
print(
    "Le coefficient de corrélation entre X et les residus vaut",
    pearsonr(X_train, residuals)[0],
    ". On a bien indépendance et donc exogénéité.",
)

### Hypothèse 3 : l'erreur suit une loi normale centrée i.e. E(ε) = 0

In [ ]:
import statistics
from scipy import stats

In [ ]:
plt.hist(residuals, density=True)

x = np.linspace(-7, 7, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))
plt.title("Histogramme des résidus en superposition avec la densité de la loi normale")
plt.show()

In [ ]:
print("La moyenne des résidus vaut", statistics.mean(residuals))
print("Mais les résidus ne suivent pas une loi normale")

In [ ]:
sm.qqplot(residuals, line="45")
print("On constate sur le qqplot que les points ne suivent pas la droite x = y")

In [ ]:
print(
    "Un test de shapiro, pour tester l'hypothèse de normalité, nous donne une pvalue de",
    stats.shapiro(residuals)[1],
    ". On rejette l'hypothèse nulle et on conclut que les résidus ne suivent pas une loi normale.",
)

### Hypothèse 4 : homoscédasticité

In [ ]:
def abline(slope, intercept):
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, "-")

In [ ]:
plt.plot(residuals, "bo")
plt.title("Nuage de points des résidus")
abline(0, 7)
abline(0, -7)

In [ ]:
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from statsmodels.compat import lzip

In [ ]:
fit = smf.ols("NumStorePurchases ~ Income", data=df_transforme).fit()

In [ ]:
bp_test = sms.het_breuschpagan(fit.resid, fit.model.exog)

In [ ]:
bp_names = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
print(lzip(bp_names, bp_test))
print("Il y a bien homoscédasticité.")

### Hypothèse 5 : absence d'autocorrélation

In [ ]:
from statsmodels.graphics.regressionplots import *

In [ ]:
sm.graphics.tsa.plot_acf(residuals)
print("On remarque une absence d'autocorrélation.")

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X, cooks[0])
plt.xlabel("Revenus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.005) if x]
print(cooks_indexes)
# affiche les indexes des individus dont les distances de cook dépassent une certaine valeur

In [ ]:
influence_plot(fit)
print("")

### Score du modèle

#### Qualité d'ajustement

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
print(f"Le R² du modèle vaut {model.score(X_train.reshape(-1, 1), Y_train)}")

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model.score(X_train.reshape(-1, 1), Y_train)}"
)
print(
    f"R² du modèle sur les données de test = {model.score(X_test.reshape(-1, 1), Y_test)}"
)

In [ ]:
Y_test_hat = model.predict(X_test.reshape(-1, 1))

In [ ]:
plt.scatter(X_test, Y_test, color="black")
plt.plot(X_test, Y_test_hat, color="red")
plt.title("Nombre d'achats en magasin en fonction du revenu : données test")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
# todo: cross-validation

## Régression linéaire multiple

Pour les besoins de la régression linéaire, nous créons une variable Childhome étant égale à la somme de Teenhome et de Kidhome.  
Nous créons également des variables muettes (dummy variables) pour inclure les variables catégoriques `Education` et `Marital_status` à la régression.

In [ ]:
df_reg = df_transforme.copy()

In [ ]:
df_reg = pd.get_dummies(df_reg, columns=["Education", "Marital_Status"])

In [ ]:
df_reg.columns

Pour cette régression, on suppose que l'entreprise veut profiler au mieux les clients qui achètent dans le magasin. C'est pourquoi la variable d'intérêt pour la régression sera le `nombre d'achats en magasin`.  

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "NbChildren",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

### Avec statsmodels

In [ ]:
model_multiple_sm = sm.OLS(Y, X).fit()

In [ ]:
model_multiple_sm.summary()

Qualité d'ajustement du modèle avant suppression de variables

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

Réduction du nombre de variables explicatives par les critères AIC / BIC

In [ ]:
def aic_OLS(X):  # entrer X le dataframe des variables explicatives
    aic_list = []
    for col in X.columns:
        X_temp = X.drop(col, axis=1)
        model_temp = sm.OLS(Y, X_temp).fit()
        aic_list.append([col, model_temp.aic])

    return aic_list

In [ ]:
def stepwise_OLS(X):
    current_aic = sm.OLS(Y, X).fit().aic  # aic avec les variables actuelles
    aic_list = aic_OLS(X)  # liste des aic par supression des variables une a une

    my_bool = 0  # pour indiquer si aucune variable n'est a enlever
    for i in range(
        len(aic_list)
    ):  # si l'aic diminue pour une variable supprimee, on enleve cette variable du df X
        if aic_list[i][1] < current_aic:
            del X[aic_list[i][0]]  # a l'interieur des crochets c'est une string
            my_bool = 1

    if my_bool == 0:
        print("Le stepwise est terminé.")

In [ ]:
stepwise_OLS(X)

In [ ]:
stepwise_OLS(X)

In [ ]:
model_multiple_sm = sm.OLS(Y, X).fit()
Y_hat = model_multiple_sm.predict(X)

In [ ]:
plt.scatter(Y, Y_hat)
plt.title("Valeurs prédites contre vraies valeurs")
abline(1, 0)

In [ ]:
model_multiple_sm.summary()

Qualité d'ajustement après suppression de variables

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

### Avec sci-kit learn : entraînement et tests sur des vraies valeurs

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
model_multiple_sk = LinearRegression().fit(X_train, Y_train)
Y_test_hat = model_multiple_sk.predict(X_test)
Y_train_hat = model_multiple_sk.predict(X_train)

In [ ]:
plt.scatter(Y_test, Y_test_hat)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

Qualité d'ajustement

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

Qualité de prédiction

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

Extraction des coefficients du modèle de régression linéaire :

In [ ]:
coefficients = pd.DataFrame(
    model_multiple_sk.coef_, X.columns, columns=["Coefficients"]
)
print(coefficients)
print("Intercept", model_multiple_sk.intercept_)

## Regression GLM

In [ ]:
import scipy

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "NbChildren",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

In [ ]:
scipy.stats.probplot(Y, dist="poisson", sparams=(5), plot=plt)
plt.show()

In [ ]:
poisson_model = sm.GLM(Y, X, family=sm.families.Poisson()).fit()

In [ ]:
poisson_model.summary()

Réduction du nombre de variables grâce au critère AIC

In [ ]:
def aic_GLM(X, Y):  # entrer X le dataframe des variables explicatives
    aic_list = []
    for col in X.columns:
        X_temp = X.drop(col, axis=1)
        model_temp = sm.GLM(Y, X_temp, family=sm.families.Poisson()).fit()
        aic_list.append([col, model_temp.aic])

    return aic_list

In [ ]:
def stepwise_GLM(X, Y):
    current_aic = (
        sm.GLM(Y, X, family=sm.families.Poisson()).fit().aic
    )  # aic avec les variables actuelles
    aic_list = aic_GLM(X, Y)  # liste des aic par supression des variables une a une

    my_bool = 0  # pour indiquer si aucune variable n'est a enlever
    for i in range(
        len(aic_list)
    ):  # si l'aic diminue pour une variable supprimee, on enleve cette variable du df X
        if aic_list[i][1] < current_aic:
            del X[aic_list[i][0]]  # a l'interieur des crochets c'est une string
            my_bool = 1

    if my_bool == 0:
        print("Le stepwise est terminé.")

In [ ]:
stepwise_GLM(X_train, Y_train)

In [ ]:
stepwise_GLM(X_train, Y_train)

In [ ]:
poisson_model = sm.GLM(Y_train, X_train, family=sm.families.Poisson()).fit()
Y_test_hat = poisson_model.predict(X_test)

In [ ]:
plt.scatter(Y, Y_hat)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

In [ ]:
print(f"MSE = {mean_squared_error(Y, Y_hat)}")
print(f"RMSE = {mean_squared_error(Y, Y_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y, Y_hat)}")

## Régression polynomiale

In [ ]:
X = df_transforme[
    [
        "Income",
        "NumWebPurchases",
        "NumDealsPurchases",
        "NumWebVisitsMonth",
        "NumCatalogPurchases",
        "MntWines",
        "MntFruits",
        "MntMeatProducts",
        "MntFishProducts",
        "MntSweetProducts",
        "MntGoldProds",
        "Year_Birth",
        "Childhome",
        "Recency",
        "NbAcceptedCampaigns",
    ]
]
Y = df_transforme["NumStorePurchases"]

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_poly, Y, test_size=0.5)

In [ ]:
# Entraîner un modèle de régression linéaire sur les données transformées
polynomial_model = LinearRegression().fit(X_train, Y_train)

In [ ]:
Y_test_hat = polynomial_model.predict(X_test)
Y_train_hat = polynomial_model.predict(X_train)

In [ ]:
plt.scatter(Y_test_hat, Y_test)
plt.title("Valeurs prédites contres vraies valeurs (valeurs test)")
abline(1, 0)

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

On constate que le modèle polynomial s'ajuste mieux aux données d'entraînement, mais est moins bon pour prédire les valeurs de test. Plus le degré des polynômes augmente, plus cet écart s'empire.

Jusqu'alors, le meilleur résultat a été obtenu par la régression linéaire multiple, avec élimination des variables par critères AIC.

# Autres modèles

## Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()

In [ ]:
lda.fit(X_train, y_train)

In [ ]:
y_pred = lda.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=lda.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=lda.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lda.classes_)
disp.plot()

## Arbre de décision

## Random Forest